# Vector DB used as Memory Demo

In [9]:
from modules.memory import Memory

In [10]:
memories = Memory(name="my_collection", db_directory="my_db_directory")

Collection 'my_collection' is ready.


# Add memories

In [11]:
# # Save some messages to the database
messages = ["Austin: My Birtday is October 24th", "Austin: I work at a car Auction as an IT Guy", "Austin: My favorite food is Steak"]
memories.save_messages_to_db(messages)

Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3
Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3
Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


Message 'Austin: My Birtday is October 24th' is already in database. Skipped.
Message 'Austin: I work at a car Auction as an IT Guy' is already in database. Skipped.
Message 'Austin: My favorite food is Steak' is already in database. Skipped.


# Query the database

In [12]:
query_text = "Austin: Do you know my birtday?"
results = memories.query_db(query_text)
print(f"Query results for '{query_text}': {results}")

Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


Query results for 'Austin: Do you know my birtday?': {'ids': [['id3011cfdb-bbca-4f4d-b565-e822b3607326', 'ida1d8a66e-2f1b-4a1e-b49b-9d467017942b', 'idbdcfd323-e1fe-47c5-a114-19041d3f7ffc']], 'embeddings': None, 'documents': [['Austin: My Birtday is October 24th', 'Austin: My favorite food is Steak', 'Austin: I work at a car Auction as an IT Guy']], 'metadatas': [[None, None, None]], 'distances': [[0.36301612854003906, 0.9841171503067017, 1.0845710039138794]]}


In [13]:
results_with_cutoff = memories.query_with_cutoff("Austin: When is my birthday?", 0.5)
results_with_cutoff

Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


'Austin: My Birtday is October 24th'

In [14]:
first_result = results['documents'][0][0]
print(f"Austin asked:\n{query_text} \nRemembered:\n{first_result}")


Austin asked:
Austin: Do you know my birtday? 
Remembered:
Austin: My Birtday is October 24th


In [15]:
def query_with_cutoff(self, query_text, cutoff):
    results = self.query_db(query_text)
    if not results or not results['distances']:
        return "Nothing found."

    closest_distance = results['distances'][0][0]
    if closest_distance > cutoff:
        return "Nothing found."

    closest_result = results['documents'][0][0]
    return closest_result


In [16]:
# import wrapper
from langchain.llms import TextGen
from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate
from langchain.chains import (
    ConversationChain
)

In [17]:
# create instance with your endpoint
llm = TextGen(model_url="https://computing-bond-il-inputs.trycloudflare.com/")

In [18]:
llm("Question: What is the capital of France?\nAnswer:")

Question: What is the capital of France?
Answer: The capital city of France is Paris.


' The capital city of France is Paris.'

In [9]:
injected_memory = ""
template = f"""You are a chatbot having a conversation with a human.
{injected_memory}
{{history}}
Austin: {{input}}
Chatbot:"""

In [10]:
#function that will query the memory object db for the user's input and return the results if found. If not found, it will return an empty string.
def memory_find(query_text):
    results = memories.query_db(query_text)
    if results:
        result = f"[Memory - {results['documents'][0][0]}]\n"
        return result
    else:
        return ""
    

In [20]:
memory = ConversationBufferMemory(input_key="history", human_prefix="Austin")

In [21]:

prompt = PromptTemplate(
    input_variables=["history","input"], template=template
)

In [22]:
 # Create a conversation chain using the channel-specific memory
conversation = ConversationChain(
    prompt=prompt,
    llm=llm,
    verbose=True,
    memory=memory,
)

In [23]:
user_input = input("Enter Message: ")
injected_memory = memory_find("Austin:" + user_input)
print(injected_memory)

Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


[Memory Austin: My Birtday is October 24th]



In [24]:
template = f"""You are a chatbot having a conversation with a human.
{injected_memory}
{{history}}
Austin: {{input}}
Chatbot:"""
prompt = PromptTemplate(
    input_variables=["history","input"], template=template
)
template

'You are a chatbot having a conversation with a human.\n[Memory Austin: My Birtday is October 24th]\n\n{history}\nAustin: {input}\nChatbot:'

## take user input

In [1]:
conversation.predict(input=user_input, stop=["\nAustin:"])

NameError: name 'conversation' is not defined